# Don't Know What to Watch?

In this notebook we'll take a look at the movies and TV shows available in our database. We'll then scour through them and use a couple of recommendation algorithms viz. conent , popularity and, collaborative filtering bases to tune a recommendaion system based on user needs. We have two datasets from MovieLens viz. 
* The Full Dataset: 26 million ratings and 750k tags applied to over 40k moviesby nearly 270k users. This dataset also includes 12 million relevance scores across 1100 tags.
* The Small Dataset: A subset of the full dataset contains 100k ratings and 1300 tags applied by 700 user to 9000 movies.

Lets build a general purpose recommendor using the Full Dataset. We'll initially do a very trivial implementation, where we'll recommend based on movie popularity. This model doesn't cater to personal taste or preference and just recommends more popular movies. So, we'll just sort movies by the ratings and put in an option to filter by genre. First we'll add all the modules and libraries required for this notebook.

In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate

import warnings;warnings.simplefilter('ignore')


We import the data from TMDB. We use their API to generate a dataset and saved as CSV in a separate utility script. We'll just import it here from input.

In [25]:
data=pd.read_csv('/kaggle/input/movie-dataset/movies_dataset.csv')
data.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/yOm993lsJyPmBodlYjgpPwBjXP9.jpg,"[35, 10751, 14]",787699,en,Wonka,Willy Wonka – chock-full of ideas and determin...,3221.716,/qhb1qOilapbapxWQn9jtRCMwXJF.jpg,2023-12-06,Wonka,False,7.195,1370
1,False,/tLxjbT5ROZRwYcpNT3nfQbqkApk.jpg,"[878, 12, 28]",609681,en,The Marvels,"Carol Danvers, aka Captain Marvel, has reclaim...",2177.632,/9GBhzXMFjgcZ3FdR9w3bUMMTps5.jpg,2023-11-08,The Marvels,False,6.394,1197
2,False,/jXJxMcVoEuXzym3vFnjqDW4ifo6.jpg,"[28, 12, 14]",572802,en,Aquaman and the Lost Kingdom,"Black Manta, still driven by the need to aveng...",2141.691,/7lTnXOy0iNtBAdRP3TZvaKJ77F6.jpg,2023-12-20,Aquaman and the Lost Kingdom,False,6.788,750
3,False,/rz8GGX5Id2hCW1KzAIY4xwbQw1w.jpg,"[35, 80]",955916,en,Lift,"An international heist crew, led by Cyrus Whit...",1113.522,/gma8o1jWa6m0K1iJ9TzHIiFyTtI.jpg,2024-01-10,Lift,False,6.407,482
4,False,/f1AQhx6ZfGhPZFTVKgxG91PhEYc.jpg,"[36, 10752, 18]",753342,en,Napoleon,An epic that details the checkered rise and fa...,1065.456,/jE5o7y9K6pZtWNNMEw3IdpHuncR.jpg,2023-11-22,Napoleon,False,6.498,1392


In [28]:
data.shape

(15500, 14)

In [27]:
data['genres'] = data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

KeyError: 'genres'